In [1]:
# Importing Libraries

import numpy as np
import torch 
import torchvision

In [2]:
import numpy as np

"""
What does the .item() means in the pytorch

Explanation: .item() is used to get the particular value from the tensor.
"""

# ================================================================== #
#                       Zero Dimenion Tensor                         #
# ================================================================== #

t0 = torch.tensor(5.0)
print(t0)
print(type(t0))

# to get the value of tensor as int/float use .item()
print(t0.item())
print(type(t0.item()))

# ================================================================== #
#                        One Dimenion Tensor                         #
# ================================================================== #

t1 = torch.tensor([1,2,3,4])
print(t1)
# How to get the particular value from 1D tensor
print(t1[2].item())

# ================================================================== #
#                        Two Dimenion Tensor                         #
# ================================================================== #

t2 = torch.tensor([[1,2],[3,4]])
print(t2[1][1].item())
print((t2.shape))

first_col = t2[:][1].data
print("First col is: ", first_col)


# ================================================================== #
#                      Tensor to numpy conversion                    #
# ================================================================== #

t2_np = t2.numpy()
print(t2_np.shape)


tensor(5.)
<class 'torch.Tensor'>
5.0
<class 'float'>
tensor([1, 2, 3, 4])
3
4
torch.Size([2, 2])
First col is:  tensor([3, 4])
(2, 2)


In [3]:
# Downloading the training data from the MNIST class

from torchvision.datasets import MNIST

dataset = MNIST(root= 'data/', train=True, download= True)
print(len(dataset))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

60000


In [4]:
# Downloading the testing data from the MNIST class

from torchvision.datasets import MNIST

test_dataset = MNIST(root= 'data/', train=False, download= True)
print(len(test_dataset))

10000


In [5]:
# transform the data from PIL images into the torch Tensor 

import torchvision.transforms as transforms

dataset = MNIST(root='data/', download = True, transform=transforms.ToTensor())

In [6]:
"""

Data preparation step:
 - training data: used to train the model
 - validation data: used to evaluate the model during training, adjusting the hyperparameters( learning rate, momentum etc)
 - test data : used to compare the different models / approaches 

"""

# random split the data into training and validation part

from torch.utils.data import random_split

train_ds, val_ds = random_split(dataset, [50000, 10000])
print(len(train_ds), len(val_ds))

50000 10000


In [7]:

"""
Data loader is used to load the data into different batches of fixed batch_size

Reason: Data has been divided into batches so that computational load would be low during model training.
Shuffling helps to introduce the randomization which speeds up the training process

"""
from torch.utils.data import DataLoader

batch_size = 128

train_batch = DataLoader(train_ds, batch_size= batch_size, shuffle= True)
val_batch = DataLoader(val_ds, batch_size= batch_size, shuffle=True)


In [8]:
# Logistic Regression Model creation without class
import torch.nn as nn 

input = 28*28*1
output_classes = 10

model = nn.Linear(in_features= input, out_features= output_classes)
print(model)


Linear(in_features=784, out_features=10, bias=True)


In [43]:
# Using class

class MNIST_sanjeev (nn.Module):

  def __init__(self, input_features, output_features):
    super().__init__()
    
    # intializing the random weight and bias matrix as per input and output features
    self.linear = nn.Linear(input_features, output_features)

  def forward_path(self, input_images):
    # reshaping the input_images (batch_size, channels, height, width)

    batch_size, channel, height, width = input_images.shape

    input_images = input_images.reshape(batch_size, channel*height*width)
    
    out = self.linear(input_images)

    return out


# creating the object of MNIST_sanjeev class  
model = MNIST_sanjeev(input, output_classes)

print(model)
print(model.linear.weight.shape)
print(model.linear.bias.shape)
print(list(model.parameters()))

MNIST_sanjeev(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)
torch.Size([10, 784])
torch.Size([10])
[Parameter containing:
tensor([[ 0.0240, -0.0229,  0.0114,  ..., -0.0003,  0.0053, -0.0020],
        [-0.0228, -0.0323,  0.0123,  ..., -0.0027,  0.0227,  0.0149],
        [-0.0325, -0.0248,  0.0267,  ...,  0.0280,  0.0304, -0.0032],
        ...,
        [ 0.0181, -0.0031, -0.0064,  ...,  0.0336, -0.0152,  0.0277],
        [-0.0148,  0.0201,  0.0251,  ...,  0.0185,  0.0198, -0.0209],
        [-0.0355,  0.0280, -0.0242,  ...,  0.0223, -0.0271,  0.0200]],
       requires_grad=True), Parameter containing:
tensor([ 0.0132, -0.0234,  0.0100,  0.0214, -0.0027, -0.0254, -0.0113, -0.0016,
         0.0151,  0.0067], requires_grad=True)]


In [44]:
for images, labels in train_batch:
  pred = model.forward_path(images)
  break

torch.Size([128, 1, 28, 28])


In [45]:
# ================================================================== #
#      Use .data to get more than 1 element from tensor              #
# ================================================================== #

print(pred.shape)
print(pred[:][:2].data)   # way to get the first two columns

torch.Size([128, 10])
tensor([[-0.2980,  0.1675,  0.0217,  0.1421,  0.1135, -0.2954, -0.0535, -0.0635,
         -0.0813,  0.0505],
        [ 0.1370,  0.1621, -0.0756,  0.0066,  0.2477, -0.3184, -0.0990,  0.3997,
          0.1083, -0.0721]])


In [34]:
# Softmax implementation using numpy

"""
Need to pass the numpy 1D vector
"""
def softmax(X):
  exp_term = np.exp(X)
  return exp_term/ np.sum(exp_term, axis=1).reshape(-1,1)


#******** very important ************
# convert pred tensor into numpy
pred_numpy = pred.data.numpy()
#******** very important ************
prob_from_numpy = softmax(pred_numpy)
print(prob_from_numpy)

# to check the sum of prob should be 1
print("Sum of prob of 0th row: ", np.sum(prob_from_numpy[0,:]))

[[0.10766796 0.08092201 0.05837158 ... 0.1253514  0.10498871 0.12326135]
 [0.09379736 0.10899425 0.11101178 ... 0.10004013 0.10068481 0.09968469]
 [0.11458891 0.0996377  0.07416312 ... 0.09456065 0.09881163 0.10425489]
 ...
 [0.131356   0.05981458 0.05707695 ... 0.10272601 0.105727   0.15296389]
 [0.09686911 0.10490798 0.08192094 ... 0.08445763 0.10346374 0.14332716]
 [0.10064989 0.11722994 0.08257543 ... 0.12264026 0.09343456 0.11268173]]
Sum of prob of 0th row:  1.0


In [39]:
# Softmax functio using torch
import torch.nn.functional as F

probs = F.softmax(pred, dim = 1)
print(probs.data) # .data is must to get the data of the tensor without grad_fn=<SoftmaxBackward0>

tensor([[0.1077, 0.0809, 0.0584,  ..., 0.1254, 0.1050, 0.1233],
        [0.0938, 0.1090, 0.1110,  ..., 0.1000, 0.1007, 0.0997],
        [0.1146, 0.0996, 0.0742,  ..., 0.0946, 0.0988, 0.1043],
        ...,
        [0.1314, 0.0598, 0.0571,  ..., 0.1027, 0.1057, 0.1530],
        [0.0969, 0.1049, 0.0819,  ..., 0.0845, 0.1035, 0.1433],
        [0.1006, 0.1172, 0.0826,  ..., 0.1226, 0.0934, 0.1127]])


In [41]:
# Evalution Metric i.e accuracy---> means out of total batch size how many images has labelled correctly by model

def accuracy(predication, lables):
  _, predicated_label = torch.max(predication, dim=1)

  # compare predicated_label with actual_label
  bool_torch = torch.eq(predicated_label, labels)
  correct_classfied_images = torch.sum(bool_torch)
  return correct_classfied_images.item()/ len(bool_torch) * 100


print(accuracy(pred, labels))

7.8125
